In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [35]:
import numpy as np
from copy import deepcopy

class PathState:
    def __init__(self,loc,steps,came_from = None):
        self.loc = tuple(loc)
        self.steps = steps
        self.came_from =came_from

    def give_neighbors(self, forest_map, slip = True ):
        neighs=[]
        nR,nC = len(forest_map),len(forest_map[0])
        
        
        cf = self.came_from
        prev = []
        while cf is not None:
            prev.append(cf.loc)
            cf=cf.came_from
            
        changes = [ (-1,0), (1,0), (0,1), (0,-1) ]
        
        is_slope = forest_map[self.loc[0]][self.loc[1]]
       
        if slip:
            if  is_slope == '>':
                changes=[ (0,1) ]
            elif  is_slope == '<':
                changes=[ (0,-1) ]
            elif  is_slope == '^':
                changes=[ (-1,0) ]
            elif  is_slope == 'v':
                changes=[ (1,0) ]
        
        for xc in changes:
            new_loc = tuple([ i+j for i,j in zip(xc,self.loc)])
        
            if (new_loc[0] > -1 and new_loc[1] > -1 
                and new_loc[0] < nR and new_loc[1] < nC
                and new_loc not in prev):
                
                if forest_map[new_loc[0]][new_loc[1]] != '#':
                    neighs.append(PathState(new_loc, self.steps+1, came_from=self))
        
        return neighs
                   
    def __repr__(self):
        return f'{self.loc[0]}-{self.loc[1]}'
    
    def __str__(self):
        return f'{self.loc[0]}-{self.loc[1]}'
    


In [22]:
from collections import deque

def perform_breadth_first(forest_map):
    frontier = deque()
    start = (0,1)
    
    frontier.append(PathState(start,0))
    
    nR, nC = len(forest_map),len(forest_map[0])
    end = (nR-1, nC-2)
    
    total_steps = []
    while len(frontier) > 0  :
        cur_loc = frontier.popleft()
    
        if cur_loc.loc == end:
            total_steps.append(cur_loc)
            continue
            
        for n_loc in cur_loc.give_neighbors(forest_map):
            frontier.append(n_loc)
                    
    return total_steps

In [29]:
def do_part_one(file_path):
    with open(file_path,'r') as f:
        forest_map = [l.strip() for l in f.readlines()]
    
    steps = perform_breadth_first(forest_map)
    
    return max([ s.steps for s  in steps])

In [30]:
%%time
tmp = do_part_one('input_data/test_23.txt')
tmp

CPU times: user 8.54 ms, sys: 1.46 ms, total: 10 ms
Wall time: 9.2 ms


94

In [31]:
%%time
do_part_one('input_data/day_23.txt')

CPU times: user 13.5 s, sys: 26.3 ms, total: 13.5 s
Wall time: 13.6 s


2170

# Part 2

In [46]:
class Node:
    def __init__(self,loc,steps):
        self.loc = tuple(loc)
        self.connected ={}
        self.steps=steps

    def add_connection(self,next_node,steps):
        self.connected[next_node.loc] = steps

In [64]:
class PathStateV2(PathState):
    
    def __init__(self,loc,steps,last_node,came_from = None):
        PathState.__init__(self,loc,steps,came_from )
        self.last_node = last_node
        
    def give_neighbors(self, forest_map):
        neighs=[]
        nR,nC = len(forest_map),len(forest_map[0])
        
        
        prev_loc = self.came_from.loc if self.came_from is not None else (-1,-1)
            
        changes = [ (-1,0), (1,0), (0,1), (0,-1) ]
        
        for xc in changes:
            new_loc = tuple([ i+j for i,j in zip(xc,self.loc)])
        
            if (new_loc[0] > -1 and new_loc[1] > -1 
                and new_loc[0] < nR and new_loc[1] < nC
                and new_loc != prev_loc):
                
                if forest_map[new_loc[0]][new_loc[1]] != '#':
                    neighs.append(PathStateV2(new_loc, self.steps+1,self.last_node, came_from=self))
        
        return neighs


In [123]:
from collections import deque

def perform_breadth_first_noslip(forest_map):
    frontier = deque()
    start = (0,1)
    graph ={ start : Node(start,0)}
    
    frontier.append(PathStateV2(start,0, graph[start]))
    
    nR, nC = len(forest_map),len(forest_map[0])
    end = (nR-1, nC-2)
    
    graph[end] = Node(end,0)
    
    while len(frontier) > 0  :
        cur_loc = frontier.popleft()
        
        neighs = cur_loc.give_neighbors(forest_map)
        if len(neighs) > 1 :
            if cur_loc.loc not in graph.keys():
                graph[cur_loc.loc] = Node(cur_loc.loc, cur_loc.steps)
        
            dist = abs(cur_loc.steps - cur_loc.last_node.steps) 
            
            graph[cur_loc.loc].add_connection(cur_loc.last_node,dist)
            graph[cur_loc.last_node.loc].add_connection(graph[cur_loc.loc],dist)
            
            for n in neighs: 
                n.last_node = graph[cur_loc.loc]
            
        for n_loc in neighs:
            
            if n_loc.loc in graph.keys():
                dist = abs(n_loc.steps - n_loc.last_node.steps) 
                
                graph[n_loc.loc].add_connection(n_loc.last_node,dist)
                graph[n_loc.last_node.loc].add_connection(graph[n_loc.loc],dist)
                
            if n_loc.loc not in graph.keys():
                frontier.append(n_loc)
                    
    return graph

In [124]:
def find_max_path(graph, cur_node, visted, end,steps=0):
    if cur_node.loc == end:
        return steps
    
    neighs = cur_node.connected.keys()
    possible = list(set(neighs) - set(visted))
    
    if len(possible) > 0 :
        return max([ find_max_path(graph, graph[n], visted +[n], end, steps+ cur_node.connected[n]) for n in possible])
    else:
        return 0

In [128]:
def do_part_two(file_path, show = False):
    with open(file_path,'r') as f:
        forest_map = [l.strip() for l in f.readlines()]
    
    graph = perform_breadth_first_noslip(forest_map)
    
    start = (0,1)
    nR, nC = len(forest_map),len(forest_map[0])
    end = (nR-1, nC-2)
    
    max_path = find_max_path(graph, graph[start], [start],end)
   
    if show : 
        forest_map = [list(l) for l in forest_map]
        for node in graph.keys():
            forest_map[node[0]][node[1]] = 'O'
            
        print('\n'.join([ ''.join(l) for l in forest_map]))
    
    
    return max_path

In [129]:
%%time
do_part_two('input_data/test_23.txt',show=True)
           

#O#####################
#.......#########...###
#######.#########.#.###
###.....#.>O>.###.#.###
###v#####.#v#.###.#.###
###O>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>O#
#.#.#v#######v###.###v#
#...#O>.#...>O>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>O>.#.>O###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################O#
CPU times: user 3.76 ms, sys: 991 µs, total: 4.75 ms
Wall time: 4.01 ms


154

In [41]:
%%time
do_part_two('input_data/day_23.txt')



KeyboardInterrupt

